Techniques/questions to look at with this particular exmaple:

+ How to merge datasets and different merge techniques
+ List of movies that are rated most by users
+ In ratings, how to address ratings sparsity problem
+ Plot distributions
+ Identify different age groups and technique of binning
+ Subgrouping and unstacking
+ Differences in ratings by gender

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/Users/suppi/Downloads/u.user', sep='|', names=u_cols,
                    encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/Users/suppi/Downloads/u.data', sep='\t', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/Users/suppi/Downloads/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

In [ ]:
users.shape

In [ ]:
users.head(10)

In [ ]:
movies.dtypes

In [ ]:
users.describe()

In [ ]:
ratings.describe()

In [ ]:
movies.head()

In [ ]:
#Selecting a subset of columns
movies[['movie_id','title']].head()

In [ ]:
#Another example of subsetting
users[users.age<40].tail(10)

In [ ]:
users[(users.age < 40) & (users.sex == 'F')].head(3)

In [ ]:
movies.head(2)

In [ ]:
ratings.head()

In [ ]:
#Merge datasets
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

In [ ]:
lens.head()

In [ ]:
#Let us answer some simple questions now. 
#Most rated movies
most_rated = lens.groupby('title').size()
most_rated.head(10)

In [ ]:
lens.title.value_counts()[:20]

In [ ]:
#Highest rated movies

highest_ratings = lens.groupby('title').agg({'rating':[np.size,np.mean]})
highest_ratings.head(10)

In [ ]:
highest_ratings.sort_values([('rating', 'mean')], ascending=False).head()

In [ ]:
atleast_100 = highest_ratings['rating']['size'] >= 200
highest_ratings[atleast_100].sort_values([('rating', 'mean')], ascending=False)[:15]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(users.age,bins=5)
plt.title("Distribution of users' ages")
plt.ylabel('count of users')
plt.xlabel('age');

In [ ]:
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
lens['age_group'] = pd.cut(lens.age, range(0, 81, 10), right=False, labels=labels)
lens['age_group'].head()

In [ ]:
lens.groupby('age_group').agg({'rating': [np.size, np.mean]})


In [ ]:
most_100 = lens.groupby('movie_id').size()[:100]

In [ ]:
lens.set_index('movie_id', inplace=True)


In [ ]:
by_age = lens.loc[most_100.index].groupby(['title', 'age_group'])
by_age.rating.mean().head(15)


In [ ]:
by_age.rating.mean().unstack(1).fillna(0)[10:20]


In [ ]:
lens.reset_index('movie_id', inplace=True)


In [ ]:
pivoted = lens.pivot_table(index=['movie_id', 'title'],
                           columns=['sex'],
                           values='rating',
                           fill_value=0)
pivoted.head()

In [ ]:
pivoted['diff'] = pivoted.M - pivoted.F
pivoted.head()

In [ ]:
pivoted.reset_index('movie_id', inplace=True)


In [ ]:
disagreements = pivoted[pivoted.movie_id.isin(most_100.index)]['diff']
disagreements.head()

In [ ]:
disagreements.describe()

In [ ]:
disagree = disagreements.copy()
disagree=disagree.sort_values()
disagree.head()

In [ ]:
disagree.plot(kind='barh', figsize=[9, 15])
plt.title('Male vs. Female Avg. Ratings\n(Difference > 0 = Favored by Men)')
plt.ylabel('Title')
plt.xlabel('Average Rating Difference');